Stand: 26.03.25

env: konstanz_karte

## Interaktive Karte

Verbinden aller bisheriger geografischen Infos: 
- Messstationen
    - DWD - Konstanz
    - Daten Schweiz
    - 12 Sensoren Konstanz
        - messstationen_meta.csv (von mir erstellt aus den Excel Dateien der Sonsordaten von Konstanz auf der Webseite)
- LCZ ??
- shape file Konstanz mit Stadtteilen
    - "Kleinräumige_Gliederung_4330766187849128049.geojson"
- Maites Pizza Infos

Theorie Grundlage:
- file:///C:/Users/majaf/OneDrive/Desktop/Konstanz/LCZ_Constance/4381f9078b2704eac4c9ab8c1631439c4ccff939_factsheet.html

In [1]:
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, HoverTool
import geopandas as gpd
import geoviews as gv
import geoviews.tile_sources as gts
import holoviews as hv
import hvplot.pandas
import pandas as pd
import panel as pn

gv.extension("bokeh")

## 1) Alle Wichtigen Daten einlesen

In [2]:
# Karte von Konstanz mit Grenzen der Stadtteile + Stadtteilnamen
konstanz_karte = gpd.read_file("maja_geodaten/Kleinräumige_Gliederung_4330766187849128049.geojson")

# Punkte der Sensorstandorte
konstanz_sensoren_pkt = pd.read_csv("maja_geodaten/messstationen_meta.csv", sep=";")

# Polygone von Stadtklimaexperte Tim - selbst gemalt von ihm und klassifiziert - Grundlage für die den LCZ Generator
# das sind alle lcz die Tim verwendet hat
layers_to_use = ['2', '5', '6', '8', '9', 'A', 'B', 'C', 'D', 'E', 'G']
# 5 und 6 und C und D haben ja die gleiche Farbe  -> muss vielleicht nochmal korrigiert werden
# lcz_colors = ['#D10000', '#FF6600', '#FF6600', '#B3B3B3', '#FFCCAA',
#               '#006A00', '#00AA00', '#B9DB79', '#B9DB79', '#000000', '#6A6AFF']

frames = []
for layer in layers_to_use:
    gdf = gpd.read_file("maja_geodaten/4381f9078b2704eac4c9ab8c1631439c4ccff939.kml",
                        driver="KML", layer=layer)
    gdf["tims_layer"] = layer  # Layerkennung hinzufügen
    frames.append(gdf)

# Alles in ein GeoDataFrame zusammenführen
tims_gdf = pd.concat(frames).reset_index(drop=True)
tims_gdf.dropna(axis=1, how='all', inplace=True)

# Farbcodes je Layer zuweisen
# color_dict = dict(zip(layers_to_use, lcz_colors))
# tims_gdf['color'] = tims_gdf['tims_layer'].map(color_dict)

# Information zu den lokalen Klimazonen - Farben, deutsch/englische label
lcz_df = pd.read_csv("maja_geodaten/lcz_data.csv")


# Testen ob alles da ist
display("Konstanz Karte:", konstanz_karte.head())
display("Konstanzer Sensoren Punkte:", konstanz_sensoren_pkt.head())
# display("Tims LCZ Polygone:", tims_gdf.head())
display("Tims Polygone:", tims_gdf.head())
print("Man könnte Tims Layer noch säubern.")
print(tims_gdf["tims_layer"].unique())
print(tims_gdf["Name"].unique())

display("Informationen zu LCZ:", lcz_df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'maja_geodaten/messstationen_meta.csv'

## 2) Karte erstellen

In [ ]:
# Join: LCZ-Daten an Tims GeoDataFrame anhängen
tims_gdf_full = tims_gdf.merge(lcz_df, left_on='tims_layer', right_on='LCZ_ID', how='left')
tims_gdf_full.head()

,Name,tessellate,extrude,visibility,geometry,tims_layer,LCZ_ID,color,label,label_deutsch
0,2,1,0,-1,"POLYGON Z ((9.17687 47.6578 0, 9.17743 47.6597...",2,2,#d10000,Compact Mid-Rise,Dicht bebaute mittelhohe Gebäude
1,2,1,0,-1,"POLYGON Z ((9.17754 47.65986 0, 9.17752 47.662...",2,2,#d10000,Compact Mid-Rise,Dicht bebaute mittelhohe Gebäude
2,2,1,0,-1,"POLYGON Z ((9.17734 47.66251 0, 9.1773 47.6643...",2,2,#d10000,Compact Mid-Rise,Dicht bebaute mittelhohe Gebäude
3,2,1,0,-1,"POLYGON Z ((9.17521 47.66678 0, 9.1752 47.6667...",2,2,#d10000,Compact Mid-Rise,Dicht bebaute mittelhohe Gebäude
4,5 unsicher,1,0,-1,"POLYGON Z ((9.19781 47.679 0, 9.20035 47.67916...",5,5,#ff6600,Open Mid-Rise,Locker bebaute mittelhohe Gebäude


In [ ]:
# Liste aller vorhandenen Layer (LCZ-Typen)
plots = []

gdf = tims_gdf_full.copy()
gdf = gdf.rename(columns={
    "tims_layer": "LCZ",
    "label_deutsch": "Bedeutung"
    })

for layer in gdf["LCZ"].unique():

    gdf_layer = gdf[gdf["LCZ"] == layer].copy()

    farbe = gdf_layer['color'].iloc[0]  # gleiche Farbe für jeden Layer

    legend_label = f"LCZ {gdf_layer["LCZ"].iloc[0]}:  {gdf_layer['Bedeutung'].iloc[0]}"
    

    p = gdf_layer.hvplot(geo=True, line_color="grey", 
                         label=legend_label, 
                         alpha=0.8, 
                         color=str(farbe)
                         )
    plots.append(p)

    # etwas geschummelt für den hover - wenn ich sonst hover_cols genutzt habe, wurde alles hellblau...
    p2 = gdf_layer.hvplot(geo=True, 
                         alpha=0,
                         hover_cols=["LCZ", "Bedeutung"]
                         ).opts(show_legend=False)
    
    plots.append(p2)

    



# Overlay aller Layer - opts() geht hier nicht
final_lcz_overlay = hv.Overlay(plots)

# Basiskarte: OpenStreetMap
osm_tile = gts.OSM.opts(alpha=0.7)

# Stadtteilgrenzen Konstanz
outline = konstanz_karte.hvplot(
    geo=True,
    fill_alpha=0, # keine Füllung
    line_width=1.5,
    line_color="grey",
    label="Stadtteilgrenze",
    hover_cols="STT_NAME", 
).opts(show_legend=False)

# Finale Karte
map_with_osm = (osm_tile * outline * final_lcz_overlay).opts(width=1000, height=700, 
                                                   title="LCZ-Zonen in Konstanz – Tims Polygone",
                                                   legend_position='top_right',
                                                   )


map_with_osm


:Overlay
   .WMTS.I                                                 :WMTS   [Longitude,Latitude]
   .Polygons.Stadtteilgrenze                               :Polygons   [Longitude,Latitude]   (STT_NAME)
   .Polygons.LCZ_2_colon_Dicht_bebaute_mittelhohe_Gebäude  :Polygons   [Longitude,Latitude]
   .Polygons.I                                             :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_5_colon_Locker_bebaute_mittelhohe_Gebäude :Polygons   [Longitude,Latitude]
   .Polygons.II                                            :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_6_colon_Locker_bebaute_Flachbauten        :Polygons   [Longitude,Latitude]
   .Polygons.III                                           :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_8_colon_Großflächige_Flachbauten          :Polygons   [Longitude,Latitude]
   .Polygons.IV                                            :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_9_colon_Locker_bebaute_Gebiete            :Polygons   [Longitude,Latitude]
   .Polygons.V                                             :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_A_colon_Dichter_Baumbestand               :Polygons   [Longitude,Latitude]
   .Polygons.VI                                            :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_B_colon_Verstreute_Bäume                  :Polygons   [Longitude,Latitude]
   .Polygons.VII                                           :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_C_colon_Sträucher_oder_Gestrüpp           :Polygons   [Longitude,Latitude]
   .Polygons.VIII                                          :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_D_colon_Niedrige_Vegetation               :Polygons   [Longitude,Latitude]
   .Polygons.IX                                            :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_E_colon_Fels_oder_versiegelte_Fläche      :Polygons   [Longitude,Latitude]
   .Polygons.X                                             :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)
   .Polygons.LCZ_G_colon_Wasserfläche                      :Polygons   [Longitude,Latitude]
   .Polygons.XI                                            :Polygons   [Longitude,Latitude]   (LCZ,Bedeutung)

In [ ]:
outline = konstanz_karte.hvplot(
    geo=True,
    fill_alpha=0,           # keine Füllung
    line_width=1.5,
    line_color="black",
    label="Stadtgrenze",
).opts(show_legend=True)

outline

:Polygons   [Longitude,Latitude]